In [1]:
from ultralytics import YOLO
import cv2

model=YOLO("yolov8n.pt")

cap=cv2.VideoCapture(0)

while True:

    ret,frame=cap.read()

    faces=model(frame)[0]
#loop over all objects detected
    for face in faces.boxes:
        x,y,x1,y1=map(int,face.xyxy[0])  #rectangle cordinates
        cls=int(face.cls[0]) #class index
        label=model.names[cls] #names is a list like [car,truck,person,....]

        if label in ['person','remote']:
            cv2.rectangle(frame,(x,y),(x1,y1),(0,255,0),2)
            cv2.putText(frame,label,(x,y-30),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)

    cv2.imshow("Person and Book detection",frame)

    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
    

cap.release()
cv2.destroyAllWindows()




0: 480x640 1 cat, 557.2ms
Speed: 19.1ms preprocess, 557.2ms inference, 38.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 296.2ms
Speed: 7.3ms preprocess, 296.2ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 261.5ms
Speed: 6.1ms preprocess, 261.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 283.7ms
Speed: 4.9ms preprocess, 283.7ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 237.6ms
Speed: 4.4ms preprocess, 237.6ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 248.2ms
Speed: 4.3ms preprocess, 248.2ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 236.0ms
Speed: 3.5ms preprocess, 236.0ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 197.5ms
Speed: 3.8ms preprocess, 197.5ms inference, 1.2ms postprocess

In [1]:
import math
def function(cx1,cy1,cx2,cy2,elapsed_time):
    dist=math.sqrt(math.pow(cx2-cx1,2)+math.pow(cy2-cy1,2))
    pixel_per_meter=8.8
    dist=dist/pixel_per_meter
    speed=(dist/elapsed_time)*3.6
    return speed

In [13]:
from ultralytics import YOLO
import cv2
from sort import Sort
import numpy as np
import time


model=YOLO("yolov8n.pt")
tracker=Sort(max_age=15,min_hits=3,iou_threshold=0.3)

cap=cv2.VideoCapture(r"C:\Users\Vansh Parmar\OneDrive\Attachments\Desktop\min\coding\Python\Opencv\Vehicle_and_its_Speed_Detection\highway_mini.mp4")
red_touch={}
prev_y={}
red_y=540
blue_y=700
while True:

    ret,frame=cap.read()
    if not ret:
        break

    cars=model(frame)[0]
#loop over all objects detected
    list=[]
    for car in cars.boxes:
        x1,y1,x2,y2=map(int,car.xyxy[0])  #rectangle cordinates
        cls=int(car.cls[0]) #class index
        label=model.names[cls] #names is a list like [car,truck,person,....]

        cv2.line(frame,(0,red_y),(1920,red_y),(0,0,250),2)
        cv2.line(frame,(0,blue_y),(1920,blue_y),(250,0,0),2)    
        
        if label in ['car','bike','truck','bus']:
            list.append([x1,y1,x2,y2])
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
         #   cv2.putText(frame,label,(x1,y1-30),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    
    track=tracker.update(np.array(list))
    
    
    for x3,y3,x4,y4,track_id in track:
        tid=int(track_id)
        cx=int((x3+x4)/2)
        cy=int((y3+y4)/2)
        
        
    #    cv2.circle(frame,(cx,cy),5,(0,0,255),2)
        prev=prev_y.get(tid,cy)
        prev_y[tid]=cy
        
    
        if prev<red_y and cy>=red_y:
            if tid not in red_touch: 
               red_touch[tid]=(cx,cy,time.time())
               cv2.circle(frame,(cx,cy),5,(0,0,255),2)

        if tid in red_touch:
               if prev<blue_y and cy>=blue_y:
                  cx1,cy1,t1=red_touch[tid]
                  print(t1)
                  elapsed_time=time.time()-t1
                  print("elapesed_time ",elapsed_time )
                  speed=function(cx1,cy1,cx,cy,elapsed_time)
                  print("speed ",speed)
                  red_touch.pop(tid)
                  cv2.putText(frame,f"{int(speed)} km/h",(int(x4),int(y3)),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)

    print("red touch",red_touch)    
    cv2.imshow("Car detection",frame)

    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
    

cap.release()
cv2.destroyAllWindows()

#243->59049
#163->26569


0: 384x640 8 cars, 2 trucks, 162.1ms
Speed: 6.9ms preprocess, 162.1ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 7 cars, 2 trucks, 235.6ms
Speed: 8.0ms preprocess, 235.6ms inference, 6.4ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 8 cars, 1 bus, 2 trucks, 245.9ms
Speed: 7.1ms preprocess, 245.9ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 7 cars, 1 bus, 2 trucks, 203.3ms
Speed: 7.7ms preprocess, 203.3ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 10 cars, 1 bus, 1 truck, 153.1ms
Speed: 5.4ms preprocess, 153.1ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 8 cars, 2 trucks, 150.2ms
Speed: 5.1ms preprocess, 150.2ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 9 cars, 2 trucks, 147.3ms
Speed: 4.7ms preprocess, 147.3ms inference, 6.1

In [ ]:
from ultralytics import YOLO
import cv2
from sort import Sort
import numpy as np
import time


model=YOLO("yolov8n.pt")
tracker=Sort(max_age=15,min_hits=3,iou_threshold=0.3)

cap=cv2.VideoCapture(r"C:\Users\Vansh Parmar\OneDrive\Attachments\Desktop\min\coding\Python\Opencv\Vehicle_and_its_Speed_Detection\carsVideo.mp4")
red_touch={}
prev_y={}
red_y=140
blue_y=280
while True:

    ret,frame=cap.read()
    if not ret:
        break

    cars=model(frame)[0]
#loop over all objects detected
    list=[]
    for car in cars.boxes:
        x1,y1,x2,y2=map(int,car.xyxy[0])  #rectangle cordinates
        cls=int(car.cls[0]) #class index
        label=model.names[cls] #names is a list like [car,truck,person,....]

        cv2.line(frame,(0,red_y),(1920,red_y),(0,0,250),2)
        cv2.line(frame,(0,blue_y),(1920,blue_y),(250,0,0),2)    
        
        if label in ['car','bike','truck','bus']:
            list.append([x1,y1,x2,y2])
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
         #   cv2.putText(frame,label,(x1,y1-30),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    
    track=tracker.update(np.array(list))
    
    
    for x3,y3,x4,y4,track_id in track:
        tid=int(track_id)
        cx=int((x3+x4)/2)
        cy=int((y3+y4)/2)
        
        
    #    cv2.circle(frame,(cx,cy),5,(0,0,255),2)
        prev=prev_y.get(tid,cy)
        prev_y[tid]=cy
        
    
        if prev<red_y and cy>=red_y:
            if tid not in red_touch: 
               red_touch[tid]=(cx,cy,time.time())
               cv2.circle(frame,(cx,cy),5,(0,0,255),2)

        if tid in red_touch:
               if prev<blue_y and cy>=blue_y:
                  cx1,cy1,t1=red_touch[tid]
                  print(t1)
                  elapsed_time=time.time()-t1
                  print("elapesed_time ",elapsed_time )
                  speed=function(cx1,cy1,cx,cy,elapsed_time)
                  print("speed ",speed)
                  red_touch.pop(tid)
                  cv2.putText(frame,f"{int(speed)} km/h",(int(x4),int(y3)),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)

    print("red touch",red_touch)    
    cv2.imshow("Car detection",frame)

    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
    

cap.release()
cv2.destroyAllWindows()

#243->59049
#163->26569


0: 384x640 5 cars, 1 bus, 179.3ms
Speed: 4.7ms preprocess, 179.3ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 5 cars, 1 bus, 253.7ms
Speed: 4.8ms preprocess, 253.7ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 5 cars, 1 bus, 174.1ms
Speed: 3.8ms preprocess, 174.1ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 4 cars, 1 bus, 1 train, 173.7ms
Speed: 3.4ms preprocess, 173.7ms inference, 6.9ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 5 cars, 1 bus, 1 train, 194.0ms
Speed: 3.1ms preprocess, 194.0ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 5 cars, 1 bus, 1 train, 189.3ms
Speed: 3.1ms preprocess, 189.3ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)
red touch {}

0: 384x640 5 cars, 1 bus, 1 train, 158.6ms
Speed: 3.1ms preprocess, 158.6ms inference, 4.2ms pos